<a href="https://colab.research.google.com/github/nao-hashi/camel/blob/master/OpenAI_Finetuning_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install openai
!pip install datasets

# OpenAI API設定

In [2]:
import os
import getpass
import openai
openai.api_key = getpass.getpass(prompt = 'OpenAIのAPIキーを入力してください··········')

OpenAIのAPIキーを入力してください····················


# 学習データの用意

In [3]:
from datasets import load_dataset

dataset = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/296 [00:00<?, ?B/s]

(…)atabricks-dolly-15k-ja-gozarinnemon.json:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [4]:
dataset["train"][0]["instruction"]

'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？'

In [5]:
dataset["train"][0]["output"]

'我、りんえもんは思う。 ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。知らんけど。'

# 学習するデータセットを用意

In [6]:
import json

list_message = []
num_data = 10
for i in range(num_data):
    instruction = dataset["train"][i]["instruction"]
    output = dataset["train"][i]["output"]
    message = [
        {"role": "user", "content": instruction},
        {
            "role": "assistant",
            "content": output,
        },
    ]
    list_message.append(message)

with open("output.jsonl", "w") as file:
    for messages in list_message:
        json_line = json.dumps({"messages": messages})
        file.write(json_line + '\n')

# jsonlファイルをアップロードする

In [7]:
upload_file = openai.File.create(
  file=open("output.jsonl", "rb"),
  purpose='fine-tune'
)

APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


# モデルを学習する

In [ ]:
start_train = openai.FineTuningJob.create(training_file=upload_file["id"],model="gpt-3.5-turbo")

In [ ]:
# List 10 fine-tuning jobs
status = openai.FineTuningJob.list(limit=10)
status["data"][0]["status"]

'succeeded'

In [ ]:
completion = openai.ChatCompletion.create(
  model=status["data"][0]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "OpenAIという会社をしっていますか？"}
  ]
)

In [ ]:
print(completion.choices[0].message["content"])

我、りんえもんは思う。知らんけど。
